# Using EcoFOCIpy to process raw field data

## DY2508 (Fall Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2025/CTDcasts/dy2508/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [4]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2508' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2508.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd013.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd014.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd015.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd029.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd030.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd031.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd034.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd035.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd040.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd041.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd043.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd044.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd046.btl
Processing /

...

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd065.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd066.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd067.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd068.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd069.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd070.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd071.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd072.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd072a.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd073.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd074.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd075.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd076.btl
Processing 

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd079.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd080.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd082.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd083.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd084.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd085.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd086.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd087.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd088.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd089.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd090.btl
Processing /Users/bell/ecoraid/2025/CTDcasts/dy2508/rawconverted/ctd091.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [6]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sal11,sal00,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sigma-t00,sigma-t11,c0ms/cm,c1ms/cm,fleco-afl,t090c,t190c,turbwetntu0,par,scan,prdm,datetime
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11
mean,31.894555,31.893064,5.522373,83.499636,240.680364,5.510336,83.317818,240.156455,24.715882,24.717364,34.017725,34.017426,0.530245,8.805318,8.803309,1.226100,0.719157,3584.454545,45.097091,2025-08-27 17:56:18.636363520
min,31.891800,31.891400,5.515200,83.391000,240.367000,5.505900,83.252000,239.964000,24.714100,24.714500,34.016474,34.013562,0.493100,8.801500,8.791000,1.206000,0.701930,3405.000000,44.839000,2025-08-27 17:56:11
25%,31.892550,31.892350,5.520750,83.472500,240.610000,5.507450,83.272000,240.030000,24.715200,24.715250,34.017266,34.017224,0.504650,8.803600,8.802200,1.214900,0.709730,3492.000000,45.018000,2025-08-27 17:56:14.500000
50%,31.894400,31.892700,5.522100,83.494000,240.668000,5.510700,83.321000,240.173000,24.715500,24.717300,34.017858,34.017366,0.516100,8.805700,8.803300,1.228500,0.721140,3575.000000,45.089000,2025-08-27 17:56:18
75%,31.895350,31.894200,5.525400,83.546000,240.812500,5.512050,83.342000,240.231500,24.717050,24.718150,34.017923,34.018469,0.547950,8.806350,8.806750,1.231400,0.728220,3677.500000,45.190000,2025-08-27 17:56:22.500000
max,31.901600,31.895200,5.526100,83.552000,240.841000,5.516600,83.414000,240.429000,24.718100,24.724800,34.019094,34.018929,0.604000,8.808700,8.808100,1.255800,0.735610,3777.000000,45.282000,2025-08-27 17:56:27
std,0.002864,0.001240,0.003421,0.051897,0.148951,0.003361,0.051817,0.146782,0.001298,0.003011,0.000793,0.001534,0.034890,0.002223,0.004972,0.015778,0.012181,122.646128,0.136130,NaN


## Create BTL report file

In [7]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [8]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2508',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2025, 8, 21),
 'EndDate': datetime.date(2025, 9, 26),
 'Project': 'Fall Mooring Cruise',
 'ChiefScientist': 'Ryan McCabe',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Fall Mooring Survey',
 'CruiseYear': 2025,
 'ctdlogs_pdf_name': 'DY2508_CruiseCastlogs.pdf',
 'Accession': '',
 'Links': '\r',
 'QC_Status': '',
 'QC_Signature': ''}

In [26]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 55303,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2508',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2508',
 'Project': 'EcoFOCI Fall Mooring',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 26.631,
 'LongitudeDeg': 164,
 'LongitudeMin': 59.211,
 'GMTDay': 28,
 'GMTMonth': 'Aug',
 'GMTYear': 2025,
 'GMTTime': 6960,
 'DryBulb': -99.9,
 'RelativeHumidity': 94,
 'WetBulb': -99.9,
 'Pressure': 1004,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 338,
 'WindSpd': 12.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 48,
 'StationNameID': 'UBS1',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': 'UBS1 - No bottle samples. tripped to test leaks. One oxy went bad on upcast - replaced',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'Chlorop

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [10]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [12]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,salinity_ch2,salinity_ch1,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,sigma_t_ch1,sigma_t_ch2,...,c1ms/cm,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,par,scan,Pressure [dbar],datetime,cast
bottle,,,,,,,,,,,,,,,,,,,,,
9.0,31.8922,31.8927,5.5152,83.391,240.367,5.5125,83.351,240.252,24.7155,24.7148,...,34.018784,0.604,8.8057,8.8075,1.2285,0.70193,3698.0,45.282,2025-08-27 17:56:23,ctd001


In [13]:
cruise_data['ctd001.btl'].columns

Index(['salinity_ch2', 'salinity_ch1', 'oxy_concM_ch1', 'oxy_percentsat_ch1',
       'oxy_conc_ch1', 'oxy_concM_ch2', 'oxy_percentsat_ch2', 'oxy_conc_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'c0ms/cm', 'c1ms/cm',
       'chlor_fluorescence', 'temperature_ch1', 'temperature_ch2', 'turbidity',
       'par', 'scan', 'Pressure [dbar]', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [14]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [16]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [17]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (bottle: 11)
Coordinates:
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/21)
    salinity_ch2        (bottle) float64 88B 31.89 31.89 31.89 ... 31.9 31.9
    salinity_ch1        (bottle) float64 88B 31.89 31.89 31.89 ... 31.89 31.9
    oxy_concM_ch1       (bottle) float64 88B 5.526 5.526 5.525 ... 5.519 5.521
    oxy_percentsat_ch1  (bottle) float64 88B 83.55 83.55 83.55 ... 83.45 83.47
    oxy_conc_ch1        (bottle) float64 88B 240.8 240.8 240.8 ... 240.5 240.6
    oxy_concM_ch2       (bottle) float64 88B 5.507 5.512 5.511 ... 5.517 5.511
    ...                  ...
    turbidity           (bottle) float64 88B 1.23 1.247 1.256 ... 1.206 1.214
    par                 (bottle) float64 88B 0.712 0.7319 ... 0.7075 0.7245
    scan                (bottle) float64 88B 3.405e+03 3.443e+03 ... 3.777e+03
    Pressure [dbar]     (bottle) float64 88B 45.16 45.05 45.07 ... 45.14 44.96
    datetime            (bottle) datetime64[ns] 88B 2025-08-27T17:56:11 ... 2...
    cast                (bottle) object 88B 'ctd001' 'ctd001' ... 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [18]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [21]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [22]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 11)
Coordinates:
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 88B 31.89...
    salinity_ch1        (latitude, longitude, time, bottle) float64 88B 31.89...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 88B 5.526...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 88B 83.55...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 88B 240.8...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 88B 5.507...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 88B 1.23 ...
    par                 (latitude, longitude, time, bottle) float64 88B 0.712...
    scan                (latitude, longitude, time, bottle) float64 88B 3.405...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 88B 45.16...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 88B ...
    cast                (latitude, longitude, time, bottle) object 88B 'ctd00...

In [23]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 11)
Coordinates:
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
  * bottle              (bottle) float64 88B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 88B 31.89...
    salinity_ch1        (latitude, longitude, time, bottle) float64 88B 31.89...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 88B 5.526...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 88B 83.55...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 88B 240.8...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 88B 5.507...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 88B 1.23 ...
    par                 (latitude, longitude, time, bottle) float64 88B 0.712...
    scan                (latitude, longitude, time, bottle) float64 88B 3.405...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 88B 45.16...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 88B ...
    cast                (latitude, longitude, time, bottle) object 88B 'ctd00...
Attributes: (12/48)
    CruiseID:                      DY2508
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-12-15T10:59:55Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [24]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

Skipping ctd072a.btl
Skipping ctd080.btl


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history